In [2]:
import os
from moviepy.editor import VideoFileClip
import numpy as np
import cv2
from glob import glob


def video2gif(video_file):
    gif_file = video_file.replace('.mp4', '.gif').replace('video', 'gif')
    clip = VideoFileClip(video_file)
    clip.speedx(2).write_gif(gif_file)
    clip.close()

def generate_video_ffmpeg(img_path, video_name, fps=30):
    if os.path.exists(video_name):
        os.system(f'rm {video_name}')
    os.system(f'ffmpeg -framerate {fps} -vsync 0 -i {img_path}' + '/%04d.png -c:v libx264 -crf 0 ' + video_name)


def generate_video(img_path, video_name, fps=15, brighten=False):
    # imgs: list of img tensors [3, H, W]
    imgs = sorted(glob(f'{img_path}/*.png'))
    imgs = [cv2.imread(img, -1) for img in imgs] # rgba
    # transfer to rgb with white background
    for i, img in enumerate(imgs):
        if img is None:
            print(f'img {i} is None')
            continue
        if img.shape[-1] == 4:
            imgs[i] = img[:, :, :3] * (img[:, :, 3:4] / 255.0) + 255 * (1 - img[:, :, 3:4] / 255.0)
            imgs[i] = imgs[i].astype(np.uint8)
    height, width = imgs[0].shape[0], imgs[0].shape[1]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(video_name, fourcc, fps, (width, height))

    for img in imgs:
        if brighten:
            img = cv2.convertScaleAbs(img, alpha=1.5, beta=1)
        video.write(img)
    video.release()

In [ ]:
dataset = 'artgs'
subset = 'sapien'
scene = 'storage_47648'
file_dir = f"../outputs/{dataset}/{subset}/{scene}/artgs/train/ours_best"
img_path = f".{file_dir}/renders/-1"
print(img_path)
video_file = f'{file_dir}/{scene}.mp4'
if not os.path.exists(video_file):
    generate_video(img_path, video_file, 30)
    video2gif(video_file)

../data/artgs/sapien/storage_47648/start/train/rgba
MoviePy - Building file ../data/artgs/sapien/storage_47648/start/train/storage_47648.gif with imageio.
